In [1]:
pip install pandas numpy ahpy scikit-learn geopy matplotlib ipywidgets


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 0.0/139.8 kB ? eta -:--:--
   -------- ------------------------------- 30.7/139.8 kB 1.4 MB/s eta 0:00:01
   ----------------- ---------------------- 61.4/139.8 kB 1.7 MB/s eta 0:00:01
   -------------------------------- ------- 112.6/139.8 kB 1.1 MB/s eta 0:00:01
   -------------------------------- ------- 112.6/139.8 kB 1.1 MB/s eta 0:00:01
   -------------------------------- ------- 112.6/139.8 kB 1.1 MB/s eta 0:00:01
   -------------------------------------- 139.8/139.8 kB 591.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/214.4 kB ? eta -:--:--
   ----------------- ---------------------- 92.2/214.4 kB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 214.4/214.4 kB 2.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   -- --


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
from ahpy import Compare
from sklearn.preprocessing import MinMaxScaler
from geopy.distance import geodesic
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load and preprocess data
def load_data():
    merged_data = pd.read_csv('cluster_with_pollution_data.csv')
    merged_data['coordinates_lon'] = merged_data['coordinates_lon'].apply(ast.literal_eval)
    merged_data['coordinates_lat'] = merged_data['coordinates_lat'].apply(ast.literal_eval)
    merged_data['speciesId'] = merged_data['speciesId'].apply(ast.literal_eval)
    return merged_data

# AHP - Assign Weights
def calculate_ahp_weights():
    criteria = {
        ('AQI_Impact', 'Species_Importance'): 3,
        ('AQI_Impact', 'Tree_Density'): 5,
        ('Species_Importance', 'Tree_Density'): 4,
    }
    comparison = Compare('Criteria', criteria, precision=3)
    weights = comparison.target_weights
    return weights

# Calculate distance functions
def calculate_distance_to_urban(row):
    urban_location = (42.0, -0.5)
    return geodesic((row['average_lat'], row['average_lon']), urban_location).km

def calculate_distance_to_water(row):
    water_location = (42.2, -0.7)
    return geodesic((row['average_lat'], row['average_lon']), water_location).km

# Filter trees based on location and radius
def filter_trees_by_location(df, input_coords, radius_km):
    df['distance_to_input'] = df.apply(
        lambda row: geodesic((row['average_lat'], row['average_lon']), input_coords).km, axis=1
    )
    return df[df['distance_to_input'] <= radius_km]

# Calculate criteria and run TOPSIS ranking
def calculate_criteria_and_rank(df, weights):
    scaler = MinMaxScaler()
    criteria_data = df[['AQI_Impact', 'Species_Importance', 'Tree_Density']]
    normalized_data = scaler.fit_transform(criteria_data)
    weighted_data = normalized_data * np.array(list(weights.values()))
    ideal_solution = np.max(weighted_data, axis=0)
    negative_ideal_solution = np.min(weighted_data, axis=0)
    dist_to_ideal = np.sqrt(np.sum((weighted_data - ideal_solution) ** 2, axis=1))
    dist_to_negative_ideal = np.sqrt(np.sum((weighted_data - negative_ideal_solution) ** 2, axis=1))
    df['TOPSIS_Score'] = dist_to_negative_ideal / (dist_to_ideal + dist_to_negative_ideal)
    df = df.sort_values(by='TOPSIS_Score', ascending=False)
    return df

# GUI Components
latitude_widget = widgets.FloatText(description="Latitude:")
longitude_widget = widgets.FloatText(description="Longitude:")
radius_widget = widgets.FloatText(description="Radius (km):")
species_id_widget = widgets.IntText(description="Species ID:")
remove_button = widgets.Button(description="Run Removal Analysis")
output = widgets.Output()

# Display Plots
def plot_all_trees(df, title="Tree Locations"):
    plt.figure(figsize=(10, 10))
    plt.scatter(df['average_lon'], df['average_lat'], c='gray', s=10, label='All Trees')
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()

# Button Callback
def on_run_button_clicked(b):
    with output:
        clear_output()
        print("Processing data...")

        # Load data and apply filtering
        merged_data = load_data()
        input_coords = (latitude_widget.value, longitude_widget.value)
        radius_km = radius_widget.value
        
        filtered_trees = filter_trees_by_location(merged_data, input_coords, radius_km)
        if filtered_trees.empty:
            print("No trees found within the specified radius.")
            return
        
        # Plot all trees in the selected area
        plot_all_trees(filtered_trees, title="Filtered Trees in Selected Area")
        
        # Calculate weights and perform AHP + TOPSIS Ranking
        weights = calculate_ahp_weights()
        ranked_trees = calculate_criteria_and_rank(filtered_trees, weights)
        
        # Display ranked trees for removal
        print("Top-ranked trees for removal based on AHP and TOPSIS:")
        display(ranked_trees[['speciesId', 'average_lon', 'average_lat', 'TOPSIS_Score']].head(10))
        
        # Highlight trees to remove on the map
        trees_to_remove = ranked_trees.head(10)
        plt.figure(figsize=(10, 10))
        plt.scatter(filtered_trees['average_lon'], filtered_trees['average_lat'], c='lightgray', s=10, label='Filtered Trees')
        plt.scatter(trees_to_remove['average_lon'], trees_to_remove['average_lat'], c='red', s=30, label='Trees for Removal')
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.title("Trees Marked for Removal in Selected Area")
        plt.legend()
        plt.grid(True)
        plt.show()

remove_button.on_click(on_run_button_clicked)

# Display Widgets
display(latitude_widget, longitude_widget, radius_widget, species_id_widget, remove_button, output)


FloatText(value=0.0, description='Latitude:')

FloatText(value=0.0, description='Longitude:')

FloatText(value=0.0, description='Radius (km):')

IntText(value=0, description='Species ID:')

Button(description='Run Removal Analysis', style=ButtonStyle())

Output()